# Robyn: Marketing Mix Modeling Application

This notebook demonstrates the usage of Robyn, a Marketing Mix Modeling (MMM) application. 
We'll go through the main steps of performing robyn_inputs and robyn_engineering.



## 1. Import Required Libraries. Define Paths.

First, be sure to setup your virtual environment. Be sure to switch over to your new environment in this notebook. 

-```cd {root_folder}```

-```python3 -m yourvenv```

-```source yourvenv/bin/activate```

-```cd Robyn/python```

-```pip install -r requirements.txt```


Then import the necessary libraries. Make sure to define your paths below.



In [1]:
import sys

# Add Robyn to path
sys.path.append("/Users/yijuilee/robynpy_release_reviews/Robyn/python/src")

In [2]:
import os
import pandas as pd
from typing import Dict
from robyn.data.entities.mmmdata import MMMData
from robyn.data.entities.enums import AdstockType
from robyn.data.entities.holidays_data import HolidaysData
from robyn.data.entities.hyperparameters import Hyperparameters, ChannelHyperparameters
from robyn.modeling.entities.modelrun_trials_config import TrialsConfig
from robyn.modeling.model_executor import ModelExecutor
from robyn.modeling.entities.enums import NevergradAlgorithm, Models
from robyn.modeling.feature_engineering import FeatureEngineering

## 2.1 Load Mock R data

We need to set the base path for the data directory.
Create a .env file in the same directory as your notebook and put in define the path to the data dir.
for example: ROBYN_BASE_PATH=.../Robyn/R/data

In [ ]:
# Read the simulated data and holidays data
dt_simulated_weekly = pd.read_csv(
    "/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/robyn/tutorials/resources/dt_simulated_weekly.csv"
)

dt_prophet_holidays = pd.read_csv(
    "/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/robyn/tutorials/resources/dt_prophet_holidays.csv"
)

## Setup MMM Data

We will now set up the MMM data specification which includes defining the dependent variable, independent variables, and the time window for analysis.

In [ ]:
def setup_mmm_data(dt_simulated_weekly) -> MMMData:

    mmm_data_spec = MMMData.MMMDataSpec(
        dep_var="revenue",
        dep_var_type="revenue",
        date_var="DATE",
        context_vars=["competitor_sales_B", "events"],
        paid_media_spends=["tv_S", "ooh_S", "print_S", "facebook_S", "search_S"],
        paid_media_vars=["tv_S", "ooh_S", "print_S", "facebook_I", "search_clicks_P"],
        organic_vars=["newsletter"],
        window_start="2016-01-01",
        window_end="2018-12-31",
    )

    return MMMData(data=dt_simulated_weekly, mmmdata_spec=mmm_data_spec)


mmm_data = setup_mmm_data(dt_simulated_weekly)
mmm_data.data.head()

## Feature Preprocessing

We will perform feature engineering to prepare the data for modeling. This includes transformations like adstock and other preprocessing steps.

In [ ]:
hyperparameters = Hyperparameters(
    {
        "facebook_S": ChannelHyperparameters(
            alphas=[0.5, 3],
            gammas=[0.3, 1],
            thetas=[0, 0.3],
        ),
        "print_S": ChannelHyperparameters(
            alphas=[0.5, 3],
            gammas=[0.3, 1],
            thetas=[0.1, 0.4],
        ),
        "tv_S": ChannelHyperparameters(
            alphas=[0.5, 3],
            gammas=[0.3, 1],
            thetas=[0.3, 0.8],
        ),
        "search_S": ChannelHyperparameters(
            alphas=[0.5, 3],
            gammas=[0.3, 1],
            thetas=[0, 0.3],
        ),
        "ooh_S": ChannelHyperparameters(
            alphas=[0.5, 3],
            gammas=[0.3, 1],
            thetas=[0.1, 0.4],
        ),
        "newsletter": ChannelHyperparameters(
            alphas=[0.5, 3],
            gammas=[0.3, 1],
            thetas=[0.1, 0.4],
        ),
    },
    adstock=AdstockType.GEOMETRIC,
    lambda_=[0, 1],
    train_size=[0.5, 0.8],
)

print("Hyperparameters setup complete.")

In [ ]:
# Create HolidaysData object
holidays_data = HolidaysData(
    dt_holidays=dt_prophet_holidays,
    prophet_vars=["trend", "season", "holiday"],
    prophet_country="DE",
    prophet_signs=["default", "default", "default"],
)
# Setup FeaturizedMMMData
feature_engineering = FeatureEngineering(mmm_data, hyperparameters, holidays_data)

In [ ]:
featurized_mmm_data = feature_engineering.perform_feature_engineering()

In [ ]:
from robyn.visualization.feature_visualization import FeaturePlotter
import matplotlib.pyplot as plt
%matplotlib inline

# Create a FeaturePlotter instance
feature_plotter = FeaturePlotter(mmm_data, hyperparameters)
# Extract the list of results
results_list = featurized_mmm_data.modNLS["results"]
# Plot spend-exposure relationship for each channel in the results
for result in results_list:
    channel = result["channel"]
    try:
        fig = feature_plotter.plot_spend_exposure(featurized_mmm_data, channel)
        plt.show()
    except ValueError as e:
        print(f"Skipping {channel}: {str(e)}")

In [ ]:
# Setup ModelExecutor
model_executor = ModelExecutor(
    mmmdata=mmm_data,
    holidays_data=holidays_data,
    hyperparameters=hyperparameters,
    calibration_input=None,  # Add calibration input if available
    featurized_mmm_data=featurized_mmm_data,
)

# Setup TrialsConfig
trials_config = TrialsConfig(iterations=54, trials=5)  # Set to the number of cores you want to use

print(
    f">>> Starting {trials_config.trials} trials with {trials_config.iterations} iterations each using {NevergradAlgorithm.TWO_POINTS_DE.value} nevergrad algorithm on x cores..."
)

# Run the model

output_models = model_executor.model_run(
    trials_config=trials_config,
    ts_validation=True,  # changed from True to False -> deacitvate
    add_penalty_factor=False,
    rssd_zero_penalty=True,
    cores=8,
    nevergrad_algo=NevergradAlgorithm.TWO_POINTS_DE,
    intercept=True,
    intercept_sign="non_negative",
    model_name=Models.RIDGE,
)
print("Model training complete.")

In [ ]:
from IPython.display import Image, display
import base64
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from robyn.visualization.model_convergence_visualizer import ModelConvergenceVisualizer

model_visualizer = ModelConvergenceVisualizer(
    moo_distrb_plot=output_models.convergence["moo_distrb_plot"],
    ts_validation_plot=output_models.convergence["ts_validation_plot"],
    moo_cloud_plot=output_models.convergence["moo_cloud_plot"],
)

# 1. Display the MOO Distribution Plot
if "moo_distrb_plot" in output_models.convergence:
    model_visualizer.display_moo_distrb_plot()

# 2. Display the MOO Cloud Plot
if "moo_cloud_plot" in output_models.convergence:
    model_visualizer.display_moo_cloud_plot()

# 3. Display time series validation and convergence plots
if "ts_validation_plot" in output_models.convergence:
    model_visualizer.display_ts_validation_plot()

In [ ]:
from robyn.tutorials.utils.data_mapper import load_data_from_json, import_input_collect, import_output_models
%load_ext autoreload
%autoreload 2

# Load data from JSON exported from R
raw_input_collect = load_data_from_json(
    "/Users/yijuilee/project_robyn/original/Robyn_original_2/Robyn/robyn_api/data/Pareto_50_InputCollect.json"
)
raw_output_models = load_data_from_json(
    "/Users/yijuilee/project_robyn/original/Robyn_original_2/Robyn/robyn_api/data/Pareto_50_OutputModels.json"
)

# Convert R data to Python objects
r_input_collect = import_input_collect(raw_input_collect)
r_output_models = import_output_models(raw_output_models)

# Extract individual components
r_mmm_data = r_input_collect["mmm_data"]
r_featurized_mmm_data = r_input_collect["featurized_mmm_data"]
r_holidays_data = r_input_collect["holidays_data"]
r_hyperparameters = r_input_collect["hyperparameters"]

In [ ]:
from anytree import Node, RenderTree
from dataclasses import is_dataclass, asdict
import pandas as pd


def build_tree(data, parent_key="", limit_trials=False, limit_repeats=False):
    """
    Recursively build a tree structure from a dictionary, list, or dataclass.

    Args:
        data: The data structure (dict, list, or dataclass) to traverse.
        parent_key: The base key path for nested keys.
        limit_trials: Whether to limit the output to the first trial.
        limit_repeats: Whether to limit the output to the first occurrence of repeated keys.

    Returns:
        A tree node representing the structure of the data.
    """
    if is_dataclass(data):
        data = asdict(data)  # Convert dataclass to dictionary

    if isinstance(data, dict):
        node = Node(f"{parent_key} (dict)")
        for key, value in data.items():
            full_key = f"{parent_key}.{key}" if parent_key else key
            child_node = build_tree(value, full_key, limit_trials, limit_repeats)
            child_node.parent = node
        return node

    elif isinstance(data, list):
        node = Node(f"{parent_key} (list, length={len(data)})")
        item_structure = None
        repeat_count = 0
        for index, item in enumerate(data):
            full_key = f"{parent_key}[{index}]"
            if limit_trials and parent_key == "trials" and index > 0:
                break

            # Serialize the structure of the item for comparison
            current_structure = (
                str({k: type(v).__name__ for k, v in item.items()}) if isinstance(item, dict) else str(type(item))
            )

            if item_structure is None:
                item_structure = current_structure
                child_node = build_tree(item, full_key, limit_trials, limit_repeats)
                child_node.parent = node
            elif current_structure == item_structure:
                repeat_count += 1
            else:
                # If the structure changes, reset the counter and process the new structure
                if repeat_count > 0:
                    summary_node = Node(f"{parent_key} (repeats for {repeat_count} times)")
                    summary_node.parent = node
                item_structure = current_structure
                repeat_count = 0
                child_node = build_tree(item, full_key, limit_trials, limit_repeats)
                child_node.parent = node

        # Add a summary node if there are repeated structures at the end
        if repeat_count > 0:
            summary_node = Node(f"{parent_key} (repeats for {repeat_count} times)")
            summary_node.parent = node

        return node

    elif isinstance(data, pd.DataFrame):
        node = Node(f"{parent_key} (DataFrame: {data.shape})")
        for column in data.columns:
            dtype = data[column].dtype
            column_node = Node(f"{parent_key}.{column} (dtype: {dtype})")
            column_node.parent = node
        return node

    else:
        dtype = type(data).__name__
        return Node(f"{parent_key} (type: {dtype})")


# Assuming featurized_mmm_data is an instance of FeaturizedMMMData
python_tree = build_tree(output_models)
r_tree = build_tree(r_output_models)

# Visualize the tree
print("Python ModelOutputs Structure:")
for pre, fill, node in RenderTree(python_tree):
    print(f"{pre}{node.name}")


print("-" * 100)
print("R ModelOutputs Structure:")
for pre, fill, node in RenderTree(python_tree):
    print(f"{pre}{node.name}")

In [ ]:
# from anytree import Node, RenderTree
# from anytree.exporter import DotExporter
# from dataclasses import is_dataclass, asdict
# import pandas as pd


# def build_tree(data, parent_key="", limit_trials=True):
#     """
#     Recursively build a tree structure from a dictionary, list, or dataclass.

#     Args:
#         data: The data structure (dict, list, or dataclass) to traverse.
#         parent_key: The base key path for nested keys.
#         limit_trials: Whether to limit the output to the first trial.

#     Returns:
#         A tree node representing the structure of the data.
#     """
#     if is_dataclass(data):
#         data = asdict(data)  # Convert dataclass to dictionary

#     if isinstance(data, dict):
#         node = Node(parent_key)
#         for key, value in data.items():
#             full_key = f"{parent_key}.{key}" if parent_key else key
#             child_node = build_tree(value, full_key, limit_trials)
#             child_node.parent = node
#         return node
#     elif isinstance(data, list):
#         node = Node(parent_key)
#         for index, item in enumerate(data):
#             if limit_trials and parent_key == "trials" and index > 0:
#                 break
#             full_key = f"{parent_key}[{index}]"
#             child_node = build_tree(item, full_key, limit_trials)
#             child_node.parent = node
#         return node
#     elif isinstance(data, pd.DataFrame):
#         node = Node(f"{parent_key} (DataFrame: {data.shape})")
#         for column in data.columns:
#             column_node = Node(f"{parent_key}.{column}")
#             column_node.parent = node
#         return node
#     else:
#         return Node(parent_key)


# # Assuming output_models and r_output_models are instances of ModelOutputs
# python_tree = build_tree(output_models)
# r_tree = build_tree(r_output_models)

# # Visualize the trees
# print("Python ModelOutputs Structure:")
# for pre, fill, node in RenderTree(python_tree):
#     print(f"{pre}{node.name}")

# print("\nR ModelOutputs Structure:")
# for pre, fill, node in RenderTree(r_tree):
#     print(f"{pre}{node.name}")

In [ ]:
import pandas as pd
import numpy as np
from typing import Dict, Any


def compare_model_values(py_output, r_output):
    """Compare key values between Python and R model outputs"""

    print("1. Basic Model Configuration Comparison:")
    basic_attrs = [
        "train_timestamp",
        "cores",
        "iterations",
        "intercept",
        "intercept_sign",
        "nevergrad_algo",
        "ts_validation",
        "add_penalty_factor",
    ]

    # Add debug prints
    print("\nDebugging attribute types:")
    for attr in basic_attrs:
        py_val = getattr(py_output, attr, None)
        r_val = getattr(r_output, attr, None)
        print(f"{attr:20s} - Python type: {type(py_val)} | R type: {type(r_val)}")
        print(f"{attr:20s} - Python value: {py_val} | R value: {r_val}")
        print("-" * 50)

    print("\n2. Trial Results Comparison (Descriptive Statistics):")
    if py_output.trials and r_output.trials:
        metrics = [
            "nrmse",
            "decomp_rssd",
            "mape",
            "rsq_train",
            "rsq_val",
            "rsq_test",
            "lambda_",
            "lambda_hp",
            "lambda_max",
            "lambda_min_ratio",
        ]
        # Convert trial results to DataFrames
        py_trials_df = pd.DataFrame(
            [{metric: getattr(trial, metric, np.nan) for metric in metrics} for trial in py_output.trials]
        )

        # Aggregate R trial metrics
        r_trials_df = pd.DataFrame(
            [
                {metric: getattr(trial, metric, pd.Series([np.nan])).mean() for metric in metrics}
                for trial in r_output.trials
            ]
        )
        # Ensure R trial data is numeric
        r_trials_df = r_trials_df.apply(pd.to_numeric, errors="coerce")
        # Calculate descriptive statistics
        py_desc = py_trials_df.describe()
        r_desc = r_trials_df.describe()
        # Print descriptive statistics
        print("\nPython Trial Descriptive Statistics:")
        print(py_desc)
        print("\nR Trial Descriptive Statistics:")
        print(r_desc)
        # Calculate and print differences
        diff_desc = py_desc - r_desc
        print("\nDifference in Descriptive Statistics:")
        print(diff_desc)

    print("\n3. Hyperparameter Comparison:")
    if hasattr(py_output, "hyper_updated") and hasattr(r_output, "hyper_updated"):
        py_hyper = py_output.hyper_updated
        r_hyper = r_output.hyper_updated

        # Find all unique keys
        all_keys = set(py_hyper.keys()) | set(r_hyper.keys())

        print("\nHyperparameter Values:")
        print(f"{'Parameter':30s} {'Python':>15s} {'R':>15s} {'Diff':>15s}")
        print("-" * 75)

        for key in sorted(all_keys):
            py_val = py_hyper.get(key, "N/A")
            r_val = r_hyper.get(key, "N/A")

            if isinstance(py_val, (int, float)) and isinstance(r_val, (int, float)):
                diff = abs(py_val - r_val)
                print(f"{key:30s} {py_val:15.6f} {r_val:15.6f} {diff:15.6f}")
            else:
                print(f"{key:30s} {str(py_val):15s} {str(r_val):15s} {'N/A':>15s}")

    print("\n4. Data Shape Comparison:")
    data_attrs = ["all_result_hyp_param", "all_x_decomp_agg", "all_decomp_spend_dist"]

    for attr in data_attrs:
        py_shape = getattr(py_output, attr).shape if hasattr(py_output, attr) else None
        r_shape = getattr(r_output, attr).shape if hasattr(r_output, attr) else None
        print(f"{attr:20s} - Python shape: {py_shape} | R shape: {r_shape}")


# Run the comparison
print("Starting detailed value comparison...\n")
compare_model_values(output_models, r_output_models)

In [ ]:
import pandas as pd

# Set display options
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.width", 1000)  # Set the display width


def get_param_dict(param):
    """Helper function to safely get parameter dictionary"""
    if hasattr(param, "__dict__"):
        return vars(param)
    elif isinstance(param, dict):
        return param
    else:
        return {}


def compare_hyperparameters(py_hyp, r_hyp):
    """Compare Hyperparameters with value previews - handles both dict and object structures"""
    print("\n=== Hyperparameters Comparison ===")
    print("\nAdstock type:")
    print(f"Python: {py_hyp.adstock}")
    print(f"R: {r_hyp.adstock}")

    # Compare hyperparameters for each channel with values
    print("\nChannel hyperparameters:")

    # Safely get channel lists
    py_channels = (
        py_hyp.hyperparameters.keys() if hasattr(py_hyp, "hyperparameters") and py_hyp.hyperparameters else []
    )
    r_channels = r_hyp.hyperparameters.keys() if hasattr(r_hyp, "hyperparameters") and r_hyp.hyperparameters else []

    # Print structure debug info
    print("\nDebug - Hyperparameters structure:")
    print(f"Python hyperparameters type: {type(py_hyp.hyperparameters)}")
    print(f"R hyperparameters type: {type(r_hyp.hyperparameters)}")

    for channel in set(py_channels) | set(r_channels):
        print(f"\nChannel: {channel}")

        # Python parameters
        if channel in py_channels:
            py_params = py_hyp.hyperparameters[channel]
            print("Python params:")
            param_dict = get_param_dict(py_params)
            for param_name, param_value in param_dict.items():
                if param_value is not None:
                    if isinstance(param_value, (list, tuple)):
                        print(f"{param_name}: {param_value[0] if param_value else None}")
                    else:
                        print(f"{param_name}: {param_value}")

        # R parameters
        if channel in r_channels:
            r_params = r_hyp.hyperparameters[channel]
            print("R params:")
            param_dict = get_param_dict(r_params)
            for param_name, param_value in param_dict.items():
                if param_value is not None:
                    if isinstance(param_value, (list, tuple)):
                        print(f"{param_name}: {param_value[0] if param_value else None}")
                    else:
                        print(f"{param_name}: {param_value}")


def describe_df_columns(df, prefix=""):
    """Helper function to describe each column in a DataFrame"""
    print(f"\n{prefix} Columns Summary:")
    for col in df.columns:
        try:
            if df[col].dtype.kind in "biufc":  # boolean, integer, unsigned integer, float, complex
                print(f"\n{col}:")
                print(df[col].describe())
        except:
            print(f"\nCould not generate statistics for {col}")


def compare_mmm_data(py_mmm_data, r_mmm_data):
    """Compare MMMData components with value previews"""
    print("\n=== MMMData Comparison ===")

    # Compare main data values
    print("\nMain data value comparison:")
    print("\nPython data summary:")
    describe_df_columns(py_mmm_data.data, "Python")
    print("\nR data summary:")
    describe_df_columns(r_mmm_data.data, "R")

    # Compare MMMDataSpec attributes
    print("\nMMMDataSpec attributes:")
    spec_attrs = [
        "all_media",
        "paid_media_spends",
        "context_vars",
        "dep_var_type",
        "rolling_window_start_which",
        "rolling_window_end_which",
        "rolling_window_length",
        "window_start",
        "window_end",
    ]

    for attr in spec_attrs:
        print(f"\n{attr}:")
        print(f"Python: {getattr(py_mmm_data.mmmdata_spec, attr)}")
        print(f"R: {getattr(r_mmm_data.mmmdata_spec, attr)}")


def compare_featurized_data(py_feat, r_feat):
    """Compare FeaturizedMMMData components with value previews"""
    print("\n=== FeaturizedMMMData Comparison ===")

    # Compare dt_mod values
    print("\ndt_mod value comparison:")
    print("\nPython dt_mod summary:")
    describe_df_columns(py_feat.dt_mod, "Python dt_mod")
    print("\nR dt_mod summary:")
    describe_df_columns(r_feat.dt_mod, "R dt_mod")

    # Compare dt_modRollWind values
    print("\ndt_modRollWind value comparison:")
    print("\nPython dt_modRollWind summary:")
    describe_df_columns(py_feat.dt_modRollWind, "Python dt_modRollWind")
    print("\nR dt_modRollWind summary:")
    describe_df_columns(r_feat.dt_modRollWind, "R dt_modRollWind")


# def compare_model_outputs(py_model, r_model):
#     """Compare ModelOutputs with value previews"""
#     print("\n=== ModelOutputs Comparison ===")

#     # Compare basic attributes
#     attrs = ["cores", "iterations", "hyper_fixed", "ts_validation"]
#     for attr in attrs:
#         if hasattr(py_model, attr) and hasattr(r_model, attr):
#             print(f"\n{attr}:")
#             print(f"Python: {getattr(py_model, attr)}")
#             print(f"R: {getattr(r_model, attr)}")

#     if py_model.trials and r_model.trials:
#         print("\n=== Last Trial Comparison ===")

#         # Compare result_hyp_param
#         print("\nResult Hyperparameter Values:")
#         if hasattr(py_model.trials[4], "result_hyp_param"):
#             print("\nPython result_hyp_param summary:")
#             describe_df_columns(py_model.trials[4].result_hyp_param, "Python")
#         if hasattr(r_model.trials[4], "result_hyp_param"):
#             print("\nR result_hyp_param summary:")
#             describe_df_columns(r_model.trials[4].result_hyp_param, "R")

#         # Compare x_decomp_agg
#         print("\nX Decomp Agg Values:")
#         if hasattr(py_model.trials[4], "x_decomp_agg"):
#             print("\nPython x_decomp_agg summary:")
#             describe_df_columns(py_model.trials[4].x_decomp_agg, "Python")
#         if hasattr(r_model.trials[4], "x_decomp_agg"):
#             print("\nR x_decomp_agg summary:")
#             describe_df_columns(r_model.trials[4].x_decomp_agg, "R")

#         # Compare decomp_spend_dist if available
#         if hasattr(py_model.trials[4], "decomp_spend_dist") and hasattr(r_model.trials[4], "decomp_spend_dist"):
#             print("\nDecomp Spend Distribution Values:")
#             print("\nPython decomp_spend_dist summary:")
#             describe_df_columns(py_model.trials[4].decomp_spend_dist, "Python")
#             print("\nR decomp_spend_dist summary:")
#             describe_df_columns(r_model.trials[4].decomp_spend_dist, "R")


def compare_model_outputs(py_model, r_model):
    """Compare ModelOutputs with value previews"""
    print("\n=== ModelOutputs Comparison ===")
    # Compare basic attributes
    attrs = ["cores", "iterations", "hyper_fixed", "ts_validation"]
    for attr in attrs:
        if hasattr(py_model, attr) and hasattr(r_model, attr):
            print(f"\n{attr}:")
            print(f"Python: {getattr(py_model, attr)}")
            print(f"R: {getattr(r_model, attr)}")
    if py_model.trials and r_model.trials:
        print("\n=== Last Trial Comparison ===")

        # Helper function to describe and stack dataframes
        def describe_and_stack(py_df, r_df, label):
            if py_df is not None and r_df is not None:
                py_desc = py_df.describe()
                r_desc = r_df.describe()
                combined = pd.concat([py_desc, r_desc], axis=1, keys=["Python", "R"])
                print(f"\n{label} summary:")
                print(combined)

        # Compare result_hyp_param
        if hasattr(py_model.trials[4], "result_hyp_param") and hasattr(r_model.trials[4], "result_hyp_param"):
            describe_and_stack(
                py_model.trials[4].result_hyp_param, r_model.trials[4].result_hyp_param, "Result Hyperparameter Values"
            )
        # Compare x_decomp_agg
        if hasattr(py_model.trials[4], "x_decomp_agg") and hasattr(r_model.trials[4], "x_decomp_agg"):
            describe_and_stack(py_model.trials[4].x_decomp_agg, r_model.trials[4].x_decomp_agg, "X Decomp Agg Values")
        # Compare decomp_spend_dist if available
        if hasattr(py_model.trials[4], "decomp_spend_dist") and hasattr(r_model.trials[4], "decomp_spend_dist"):
            describe_and_stack(
                py_model.trials[4].decomp_spend_dist,
                r_model.trials[4].decomp_spend_dist,
                "Decomp Spend Distribution Values",
            )


def run_comprehensive_comparison(py_data, r_data):
    """Run all comparisons with value previews"""
    print("Starting comprehensive data comparison...")

    # compare_mmm_data(py_data["mmm_data"], r_data["mmm_data"])
    # compare_featurized_data(py_data["featurized_mmm_data"], r_data["featurized_mmm_data"])
    # compare_hyperparameters(py_data["hyperparameters"], r_data["hyperparameters"])
    compare_model_outputs(py_data["model_outputs"], r_data["model_outputs"])


# Your existing data loading code remains the same
python_data = {
    "mmm_data": mmm_data,
    "featurized_mmm_data": featurized_mmm_data,
    "hyperparameters": hyperparameters,
    "model_outputs": output_models,
}

r_data = {
    "mmm_data": r_mmm_data,
    "featurized_mmm_data": r_featurized_mmm_data,
    "hyperparameters": r_hyperparameters,
    "model_outputs": r_output_models,
}

run_comprehensive_comparison(python_data, r_data)

In [ ]:
print("Python")
print(hyperparameters.hyperparameters)

print("R")
print(r_input_collect["hyperparameters"].hyperparameters)

In [ ]:
# def compare_x_decomp_agg(py_model, r_model):
#     """Compare x_decomp_agg structure and values between Python and R"""
#     print("\nComparing x_decomp_agg:")
#     print("Python columns:", py_model.all_x_decomp_agg.columns.tolist())
#     print("R columns:", r_model.all_x_decomp_agg.columns.tolist())

#     # Compare common columns statistics
#     common_cols = list(set(py_model.all_x_decomp_agg.columns) & set(r_model.all_x_decomp_agg.columns))
#     print("\nCommon columns statistics:")
#     for col in common_cols:
#         py_stats = py_model.all_x_decomp_agg[col].describe()
#         r_stats = r_model.all_x_decomp_agg[col].describe()
#         print(f"\n{col}:")
#         print("Python:", py_stats)
#         print("R:", r_stats)


# def compare_decomp_spend_dist(py_model, r_model):
#     """Compare decomp_spend_dist structure and values between Python and R"""
#     print("\nComparing decomp_spend_dist:")
#     print("Python columns:", py_model.all_decomp_spend_dist.columns.tolist())
#     print("R columns:", r_model.all_decomp_spend_dist.columns.tolist())

#     # Compare value distributions for key metrics
#     key_metrics = ["effect_share", "spend_share", "roi_total"]
#     print("\nKey metrics distributions:")
#     for metric in key_metrics:
#         if metric in py_model.all_decomp_spend_dist.columns and metric in r_model.all_decomp_spend_dist.columns:
#             print(f"\n{metric}:")
#             print("Python:", py_model.all_decomp_spend_dist[metric].describe())
#             print("R:", r_model.all_decomp_spend_dist[metric].describe())


# def compare_media_effect_calculations(py_model, r_model):
#     """Compare media effect calculations between Python and R"""
#     # Compare unique media channels
#     py_media = py_model.all_decomp_spend_dist["rn"].unique()
#     r_media = r_model.all_decomp_spend_dist["rn"].unique()

#     print("\nMedia channels comparison:")
#     print("Python channels:", sorted(py_media))
#     print("R channels:", sorted(r_media))

#     # Compare aggregated effects per channel
#     print("\nAggregated effects per channel:")
#     py_effects = py_model.all_decomp_spend_dist.groupby("rn")["xDecompAgg"].mean()
#     r_effects = r_model.all_decomp_spend_dist.groupby("rn")["xDecompAgg"].mean()

#     print("\nPython effects:")
#     print(py_effects)
#     print("\nR effects:")
#     print(r_effects)


# # Run comparisons
# print("Running detailed comparisons between Python and R outputs...")
# compare_x_decomp_agg(output_models, r_output_models)
# compare_decomp_spend_dist(output_models, r_output_models)
# compare_media_effect_calculations(output_models, r_output_models)

In [ ]:
def print_data_structure(data):
    print("Columns:", data.columns.tolist())
    print("\nFirst row:", data.iloc[0].to_dict())
    print("\nShape:", data.shape)


# Assuming you want to print the structure for the first trial
first_trial_r = r_output_models.trials[0].decomp_spend_dist
first_trial_python = output_models.trials[0].decomp_spend_dist

print("R exported data structure:")
print_data_structure(first_trial_r)

# With Python calculated data
print("\nPython calculated data structure:")
print_data_structure(first_trial_python)